# AI-generated CV testing

## Data retreival

In [3]:
import pandas as pd
import psycopg2

In [18]:
with psycopg2.connect(
    dbname="cv_manager",
    user="cvuser",
    password="cvpassword",
    host="localhost",
    port='5432'
) as conn:
    query = "SELECT * FROM experiences;"
    data = pd.read_sql(query, conn)

/var/folders/qj/506gdrfn55jc0hchzk5d05540000gn/T/ipykernel_23856/192697586.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


Let's create a CV for user 1. 

In [57]:
# let's create a CV for user 1
query = """
SELECT 
    e.organisation,
    e.title as engagement_title,
    ex.type,
    ex.title as experience_title,
    ex.description
FROM 
    users u
LEFT JOIN engagements e on u.user_id = e.user_id
LEFT JOIN experiences ex on e.engagement_id = ex.engagement_id
where u.user_id=1;
"""
with psycopg2.connect(
    dbname="cv_manager",
    user="cvuser",
    password="cvpassword",
    host="localhost",
    port='5432'
) as conn:
    data = pd.read_sql(query, conn)

data

/var/folders/qj/506gdrfn55jc0hchzk5d05540000gn/T/ipykernel_23856/2523801472.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


,organisation,engagement_title,type,experience_title,description
0,Google,Software Engineer,work,Software Engineer,Situation: Google was scaling its cloud servic...
1,Google,Software Engineer,work,Software Engineer,Situation: Google needed a secure authenticati...
2,McDonald’s,Kitchen Hand,work,Kitchen Hand,Situation: High customer volume at McDonald's....
3,McDonald’s,Kitchen Hand,work,Kitchen Hand,Situation: Frequent errors in order fulfillmen...
4,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Developed custom web applications f...
5,Self-employed,Software Engineer,side-hustle,Software Engineer,Situation: Clients needed better data insights...
6,MIT,Computer Science BSc,education,Computer Science BSc,Situation: Studying foundational computer scie...


# CV generation

Let's create a crew of agents who can generate a CV. First, let's build an agent which can, when given a job description, identify the most relevant experiences from a master CV.

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio",
    streaming=True
)

In [54]:
role = "Senior Software Engineer"

job_requirements = """
What it's all about…

As a Engineering Tech Lead, you will be responsible for:

Define and communicate technical strategies aligning with business strategies
Identify, and evaluate technical solutions to drive product and ecommerce vision
Identify and implement new technologies, tools and processes to enhance efficiency and product quality
Design and develop scalable and efficient systems
Optimise system performance, enhance security and maintain high-quality codebases
Implement and manage CI/CD pipelines for streamlined software development
Conduct code reviews, establish coding standards and best practices
Lead, mentor and inspire a team of engineers
Encourage collaboration and continuous improvement
Represent the engineering team and contribute to planning and decision making
Ensure seamless integration across frontend and backend components
Collaborate on database scaling and optimisation using technologies such as MySQL and PlanetScale
Work closely with the Product Manager to define and prioritise the roadmap ensuring technical deliverables
What we will need from you…

Previous experience working in a frontend and backend software engineering role
Proven experience in leading a technical team and mentoring engineers
Strong proficiency in JavaScript, TypeScript and modern web frameworks (React, Node.js)
Solid understanding of web service architectures (REST, GraphQL)
Experience with web analytics platforms (GA4, Google Tag Manager) and tracking strategies
Experience with cloud-based infrastructure (Fly.io, AWS) and container technologies (Docker)
Experience with Agile methodologies and collaborative tools (Notion, JIRA, Trello)
Demonstrated success in delivering and optimising digital products
Knowledge in testing strategies, performance optimisation and DevOps principles
A passion for continuous learning and driving technical innovation
Excellent problem-solving skills and attention to detail
Strong communication skills
"""

In [55]:
from langchain_core.prompts import ChatPromptTemplate

selector_prompt = """
You are a professional career coach. You will be given a master CV and details of the role which the user is applying for.
Your job is to identify the top 3 most relevant work experiences and explain why. 
Return the indexes of the experiences you have chosen along with reasoning for each experience.

Role: {role}

Job Requirements: {job_requirements}

Master CV: {master_cv}

Output:
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", selector_prompt)]
)

In [59]:
prompt = prompt_template.invoke({"role": role, "job_requirements": job_requirements, "master_cv": data})
print(prompt.to_messages()[0].content)


You are a professional career coach. You will be given a master CV and job description.
Your job is to identify the top 3 most relevant work experiences and explain why. 
Your output should be a filtered down master CV with the experiences you have chosen along with an extra field for your reasoning.

Role: Senior Software Engineer

Job Requirements: 
What it's all about…

As a Engineering Tech Lead, you will be responsible for:

Define and communicate technical strategies aligning with business strategies
Identify, and evaluate technical solutions to drive product and ecommerce vision
Identify and implement new technologies, tools and processes to enhance efficiency and product quality
Design and develop scalable and efficient systems
Optimise system performance, enhance security and maintain high-quality codebases
Implement and manage CI/CD pipelines for streamlined software development
Conduct code reviews, establish coding standards and best practices
Lead, mentor and inspire a te

In [60]:
chain = prompt_template | llm

In [61]:
response = chain.invoke({"role": role, "job_requirements": job_requirements, "master_cv": data})
print(response.content)

Based on the provided job description and master CV, I have filtered down the top 3 most relevant work experiences as follows:

**Filtered Down Master CV:**

| Organisation | Engagement Title | Type | Experience Title | Reasoning |
| --- | --- | --- | --- | --- |
| Google | Software Engineer | work | Software Engineer | Leadership, technical expertise |
| Self-employed | Software Engineer | side-hustle | Software Engineer | Custom web application development, problem-solving |
| MIT | Computer Science BSc | education | Computer Science BSc | Foundational knowledge in computer science, problem-solving skills |

**Rationale:**

1. **Google (2 instances)**: As a Senior Software Engineer, experience leading technical teams and communicating strategies will be highly valued. Google's scale and security requirements also demonstrate the candidate's ability to handle complex systems.
2. **Self-employed (2 instances)**: Developing custom web applications and data insights for clients showcases

In [45]:
from langchain_core.prompts import ChatPromptTemplate

summarization_prompt_template = """
You are a professional CV writer. Given the following detailed STAR-formatted work experience and a set of job requirements, generate a concise, punchy bullet point suitable for a CV.

Job Requirements: {job_requirements}

Experience in STAR format:
Situation: {situation}
Task: {task}
Action: {action}
Result: {result}

Bullet Point:
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template)]
)

In [50]:
print(llm.invoke(prompt))

content="<think>\nOkay, so I need to help improve the user's CV excerpt based on the job requirements provided. Let me start by understanding what each part of the job entails.\n\nThe job has six main points:\n1. Propecting & Lead Generation: Actively seeking new clients through networking and cold calling.\n2. Client Relationship Management: Building and nurturing relationships with clients.\n3. Property Listings: Securing new property listings by presenting compelling pitches.\n4. Conducting Viewings: Organizing and leading property viewings to attract buyers.\n5. Negotiating Sales: Skillfully negotiating offers between buyers and sellers.\n6. Market Analysis: Staying updated on market trends, values, and competitors.\n\nLooking at the user's CV excerpt: They spoke to many clients, some were nice, some weren't. They had to show people around properties and tell them about the place. It was alright, but sometimes it was a bit boring.\n\nFirst, I notice that the job emphasizes networki

In [51]:
print("""I have successfully established relationships with several clients over the past year, which has been crucial in building long-term partnerships. While my initial interactions were engaging, I also recognized the importance of maintaining a balance between networking and client engagement to avoid feeling monotonous. \n\nTo promote new property listings, I’ve consistently sought out properties that align with the client’s values, ensuring they showcase the best features without overwhelming them. This approach has kept the sales process smooth and positive for clients. Additionally, I have actively conducted viewings to highlight key aspects of each property, fostering a collaborative and positive sales experience.\n\nI’ve also been meticulous in maintaining strong client relationships, consistently striving to exceed client expectations and providing value that exceeds what was initially offered. By staying updated on market trends and competitor activities, I ensure my clients receive accurate and relevant information.\n\nFinally, I excel at negotiation skills, ensuring each transaction results in the best possible outcome for all parties involved. My proactive approach has been instrumental in delivering exceptional value to clients.""")

I have successfully established relationships with several clients over the past year, which has been crucial in building long-term partnerships. While my initial interactions were engaging, I also recognized the importance of maintaining a balance between networking and client engagement to avoid feeling monotonous. 

To promote new property listings, I’ve consistently sought out properties that align with the client’s values, ensuring they showcase the best features without overwhelming them. This approach has kept the sales process smooth and positive for clients. Additionally, I have actively conducted viewings to highlight key aspects of each property, fostering a collaborative and positive sales experience.

I’ve also been meticulous in maintaining strong client relationships, consistently striving to exceed client expectations and providing value that exceeds what was initially offered. By staying updated on market trends and competitor activities, I ensure my clients receive ac

In [39]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

/opt/anaconda3/envs/cv_env/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1382: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


{'setup': 'the setup of the joke', 'punchline': 'the punchline to the joke'}